# Lab 4 Reproduction: Sparse Matrix Multiplication and Hardware Optimization

Reproduces the key results from Lab 4 (Parts 1–5) using AccelForge.

**Architecture:** BackingStorage (DRAM) → Buffer (regfile) → MAC  
**Workload:** SpMSpM Z[m,n] = A[m,k] * B[k,n], M=K=N=8  
**Default densities:** A=0.25, B=0.5  
**Sparse configs:** Gating, Skipping (CSR), Compressed-only (CSR)

In [ ]:
import os
import sys
import math
import tempfile

import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add accelforge to path
REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, REPO_ROOT)

from accelforge.frontend.spec import Spec
from accelforge.model.main import evaluate_mapping

LAB4_DIR = os.path.join(REPO_ROOT, 'tests', 'input_files', 'lab4')
print(f'Using configs from: {LAB4_DIR}')

## 1. Configuration Files

Lab 4 uses a 2-level memory hierarchy (DRAM → Buffer → MAC) with an untiled mapping.
All loops are at the Buffer level with loop order N → K → M (outer to inner).

In [ ]:
for name in ['arch.yaml', 'workload.yaml', 'mapping.yaml']:
    with open(os.path.join(LAB4_DIR, name)) as f:
        print(f'=== {name} ===')
        print(f.read())
        print()

In [ ]:
for name in ['sparse_gating.yaml', 'sparse_skipping.yaml', 'sparse_compressed.yaml']:
    with open(os.path.join(LAB4_DIR, name)) as f:
        print(f'=== {name} ===')
        print(f.read())
        print()

## 2. Helper Functions

In [ ]:
M = K = N = 8


def make_workload_yaml(density_a=0.25, density_b=0.5, m=8, k=8, n=8):
    """Generate workload dict with given densities."""
    return {
        'workload': {
            'iteration_space_shape': {
                'm': f'0 <= m < {m}',
                'n': f'0 <= n < {n}',
                'k': f'0 <= k < {k}',
            },
            'bits_per_value': {'All': 8},
            'einsums': [{
                'name': 'SpMSpM',
                'tensor_accesses': [
                    {'name': 'A', 'projection': ['m', 'k'], 'density': density_a},
                    {'name': 'B', 'projection': ['n', 'k'], 'density': density_b},
                    {'name': 'Z', 'projection': ['m', 'n'], 'output': True},
                ],
            }],
        }
    }


def run_lab4(sparse_yaml=None, density_a=None, density_b=None):
    """Run a Lab 4 configuration and return the result.
    
    Uses default workload (d_A=0.25, d_B=0.5) unless overridden.
    """
    files = [os.path.join(LAB4_DIR, 'arch.yaml')]
    
    if density_a is not None or density_b is not None:
        da = density_a if density_a is not None else 0.25
        db = density_b if density_b is not None else 0.5
        wl = make_workload_yaml(da, db)
        with tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False) as f:
            yaml.dump(wl, f)
            files.append(f.name)
    else:
        files.append(os.path.join(LAB4_DIR, 'workload.yaml'))
    
    files.append(os.path.join(LAB4_DIR, 'mapping.yaml'))
    if sparse_yaml:
        files.append(os.path.join(LAB4_DIR, sparse_yaml))
    
    spec = Spec.from_yaml(*files)
    return evaluate_mapping(spec)


def get_energy(result):
    """Get total energy in pJ."""
    return float(result.data['Total<SEP>energy'].iloc[0])


def get_cycles(result):
    """Get total latency in cycles."""
    return float(result.data['Total<SEP>latency'].iloc[0])


def get_component_energy(result, component):
    """Get per-component energy."""
    energy = result.energy(per_component=True)
    return float(energy.get(component, 0))


def get_component_latency(result, component):
    """Get per-component latency."""
    for col in result.data.columns:
        if col.endswith(f'latency<SEP>{component}'):
            return float(result.data[col].iloc[0])
    return 0.0

---

# Part 1: Sparse Optimization Opportunities

Matrix multiplication $Z_{m,n} = A_{m,k} \cdot B_{k,n}$ with M=K=N=8.

**Key concepts:**
- **Effectual** multiply: both operands nonzero (contributes to output)
- **Ineffectual** multiply: at least one operand is zero (wasted work)
- **Gating:** hardware stays idle on ineffectual ops → saves **energy only**
- **Skipping:** hardware fast-forwards to next effectual op → saves **energy + latency**

In [ ]:
print('=== Question 1.2–1.4: Effectual Operations ===')
total = M * K * N
print(f'Total multiplies (M=K=N={M}): {total}')
print()

# Q1.2: d_A=1, d_B=1
eff_12 = int(total * 1.0 * 1.0)
print(f'Q1.2: d_A=1.0, d_B=1.0 \u2192 effectual = {eff_12}')

# Q1.3: d_A=0.5, d_B=1.0
eff_13 = int(total * 0.5 * 1.0)
ineff_13 = total - eff_13
print(f'Q1.3: d_A=0.5, d_B=1.0 \u2192 effectual = {eff_13}, ineffectual = {ineff_13}')

# Q1.4: d_A=0.5, d_B=0.5
eff_14 = int(total * 0.5 * 0.5)
print(f'Q1.4: d_A=0.5, d_B=0.5 \u2192 effectual = {eff_14}')

print()
print('=== Question 1.5: Gating vs Skipping ===')
print('Gating  saves: energy only')
print('Skipping saves: energy + latency (both)')

print()
print('=== Question 1.7: Compression overhead ===')
print('False: compression metadata can exceed savings at high density.')

---

# Part 2: Saving Energy with Gating

Gating at Buffer and MAC: Z is gated conditioned on [A, B].
No compression — tensors stored uncompressed.

With d_A=0.25, d_B=0.5: P(effectual) = 0.125, so 87.5% of Z reads/writes at Buffer are gated.

**Sparseloop reference (Q2.1):**
- Dense fJ/Alg-Compute: 7047.25
- Gated fJ/Alg-Compute: 3972.35
- Gating saves energy, no impact on latency

In [ ]:
# Run dense and gating configs
dense_result = run_lab4()
gating_result = run_lab4('sparse_gating.yaml')

dense_energy = get_energy(dense_result)
dense_cycles = get_cycles(dense_result)
gating_energy = get_energy(gating_result)
gating_cycles = get_cycles(gating_result)

alg_computes = M * K * N  # 512
eff_computes = int(alg_computes * 0.25 * 0.5)  # 64

print('=== Part 2: Dense vs Gating ===')
print(f'Algorithmic computes: {alg_computes}')
print(f'Effectual computes (d_A=0.25, d_B=0.5): {eff_computes}')
print()

print(f'{"":>20} {"Dense":>12} {"Gating":>12} {"SL Dense":>12} {"SL Gating":>12}')
print('-' * 70)
print(f'{"Total energy (pJ)":>20} {dense_energy:>12.2f} {gating_energy:>12.2f}')
print(f'{"fJ/Alg-Compute":>20} {dense_energy*1000/alg_computes:>12.2f} '
      f'{gating_energy*1000/alg_computes:>12.2f} '
      f'{7047.25:>12.2f} {3972.35:>12.2f}')
print(f'{"Total cycles":>20} {dense_cycles:>12.0f} {gating_cycles:>12.0f}')
print()

print('Per-component energy (pJ):')
for comp in ['BackingStorage', 'Buffer', 'MAC']:
    de = get_component_energy(dense_result, comp)
    ge = get_component_energy(gating_result, comp)
    delta = ge - de
    print(f'  {comp:>20}: {de:>10.2f} \u2192 {ge:>10.2f}  ({delta:+.2f} pJ)')

print()
print('Q2.1 Answers:')
print(f'  Which storage element was gated? Buffer')
print(f'  Which compute element was gated? MAC')
print(f'  Gating did NOT change energy of: BackingStorage (DRAM)')
print(f'  Gating impact on latency: no impact ({dense_cycles:.0f} \u2192 {gating_cycles:.0f} cycles)')
print(f'  Gating impact on energy: decreases ({dense_energy:.2f} \u2192 {gating_energy:.2f} pJ)')

---

# Part 3: Skipping and Compression

CSR (UOP+CP) compression at both BackingStorage and Buffer, with skipping SAF.
Skipping targets A (conditioned on B), B (conditioned on A), and Z (conditioned on A, B).

**Sparseloop reference (Q3.1):**
- Dense fJ/compute = 7047.25
- Gated fJ/alg-compute = 3972.35, fJ/compute = 31778.79
- Skipped fJ/alg-compute = 1919.80, fJ/compute = 15358.43

In [ ]:
# Run skipping and compressed-only configs
skipping_result = run_lab4('sparse_skipping.yaml')
compressed_result = run_lab4('sparse_compressed.yaml')

skip_energy = get_energy(skipping_result)
skip_cycles = get_cycles(skipping_result)
comp_energy = get_energy(compressed_result)
comp_cycles = get_cycles(compressed_result)

print('=== Part 3: Dense vs Gating vs Skipping ===')
print(f'{"":>22} {"Dense":>12} {"Gating":>12} {"Compressed":>12} {"Skipping":>12}')
print('-' * 74)
print(f'{"Total energy (pJ)":>22} {dense_energy:>12.2f} {gating_energy:>12.2f} '
      f'{comp_energy:>12.2f} {skip_energy:>12.2f}')
print(f'{"fJ/Alg-Compute":>22} {dense_energy*1000/alg_computes:>12.2f} '
      f'{gating_energy*1000/alg_computes:>12.2f} '
      f'{comp_energy*1000/alg_computes:>12.2f} '
      f'{skip_energy*1000/alg_computes:>12.2f}')
print(f'{"fJ/Compute":>22} {dense_energy*1000/alg_computes:>12.2f} '
      f'{gating_energy*1000/eff_computes:>12.2f} '
      f'{comp_energy*1000/eff_computes:>12.2f} '
      f'{skip_energy*1000/eff_computes:>12.2f}')
print(f'{"Total cycles":>22} {dense_cycles:>12.0f} {gating_cycles:>12.0f} '
      f'{comp_cycles:>12.0f} {skip_cycles:>12.0f}')

print()
print('Per-component energy (pJ):')
for comp in ['BackingStorage', 'Buffer', 'MAC']:
    de = get_component_energy(dense_result, comp)
    ge = get_component_energy(gating_result, comp)
    ce = get_component_energy(compressed_result, comp)
    se = get_component_energy(skipping_result, comp)
    print(f'  {comp:>20}: Dense={de:>8.2f}  Gate={ge:>8.2f}  Comp={ce:>8.2f}  Skip={se:>8.2f}')

print()
print('Sparseloop reference (fJ/Alg-Compute):')
print(f'  Dense: 7047.25, Gated: 3972.35, Skipped: 1919.80')

print()
print('Key observations:')
print(f'  Gating: reduces energy (Buffer + MAC), no change in latency')
print(f'  Compressed: reduces BackingStorage energy (fewer data accesses), no SAF')
print(f'  Skipping: reduces all components (fewer accesses + fewer cycles)')
print(f'  Latency: Dense={dense_cycles:.0f}, Gating={gating_cycles:.0f}, '
      f'Compressed={comp_cycles:.0f}, Skipping={skip_cycles:.0f}')

### Q3.2: Effect of increased sparsity

- Increased sparsity (more zeros) **decreases** total energy with gating/skipping
- Increased sparsity **increases** fJ/compute with skipping (fewer computes, but metadata overhead is fixed)
- Increased sparsity **decreases** fJ/algorithmic-compute with skipping

### Q3.3: Density Sweep with Skipping

In [ ]:
density_A, density_B, pJ_algo, pJ_actual = [], [], [], []

print(f'{"d_A":>6} {"d_B":>6} {"fJ/Alg-Comp":>14} {"fJ/Compute":>14} '
      f'{"Energy(pJ)":>12} {"Cycles":>8}')
print('-' * 68)

for da in [0.25, 0.5, 0.75]:
    for db in [0.25, 0.5, 0.75]:
        r = run_lab4('sparse_skipping.yaml', density_a=da, density_b=db)
        e = get_energy(r)
        c = get_cycles(r)
        eff = max(1, int(alg_computes * da * db))
        fj_alg = e * 1000 / alg_computes
        fj_comp = e * 1000 / eff
        
        density_A.append(da)
        density_B.append(db)
        pJ_algo.append(fj_alg)
        pJ_actual.append(fj_comp)
        
        print(f'{da:6.2f} {db:6.2f} {fj_alg:14.2f} {fj_comp:14.2f} '
              f'{e:12.2f} {c:8.0f}')

In [ ]:
energy_per_compute_df = pd.DataFrame({
    'density_a': density_A,
    'density_b': density_B,
    'fJ_algorithmic': pJ_algo,
    'fJ_actual': pJ_actual,
})

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

ax = sns.heatmap(
    energy_per_compute_df.pivot(index='density_a', columns='density_b', values='fJ_algorithmic'),
    annot=True, fmt='.0f', ax=axes[0], cmap='YlOrRd'
)
ax.set_title('fJ / Algorithmic-Compute')
ax.set_xlabel('B density')
ax.set_ylabel('A density')

ax = sns.heatmap(
    energy_per_compute_df.pivot(index='density_a', columns='density_b', values='fJ_actual'),
    annot=True, fmt='.0f', ax=axes[1], cmap='YlOrRd'
)
ax.set_title('fJ / Compute')
ax.set_xlabel('B density')
ax.set_ylabel('A density')

fig.suptitle('Q3.3: Energy per Operation at Different Densities (Skipping + CSR)', fontsize=13)
plt.tight_layout()
plt.show()

---

# Part 4: Effect of Sparsity on Compressed Tensor Buffer Space Usage

With CSR compression (UOP+CP) at Buffer, the storage for tensor A is split into:
- **Data storage:** the actual nonzero values
- **Format storage:** UOP offset array (M+1 entries) + CP coordinate metadata

**Sparseloop reference (Q4.1):**

| Density | Data | Format | Combined |
|---------|------|--------|----------|
| 0.2     | 13   | 25     | 38       |
| 0.4     | 26   | 41     | 67       |
| 0.6     | 39   | 49     | 88       |
| 0.8     | 52   | 65     | 117      |
| 1.0     | 64   | 73     | 137      |

Uncompressed A = 64 words. Compression is beneficial below density ~0.4.

In [ ]:
DENSITIES_PART4 = [0.2, 0.4, 0.6, 0.8, 1.0]
SL_DATA = [13, 26, 39, 52, 64]
SL_FORMAT = [25, 41, 49, 65, 73]

# Analytical computation of CSR storage at Buffer for tensor A
# A has M=8 rows, K=8 columns. CSR at Buffer: 2 ranks (UOP over M, CP over K).
# UOP payload = M+1 = 9 entries (offset array for each row + sentinel)
# CP metadata = M * ceil(d * K) coordinates (one per nonzero per row)
# Data storage = ceil(d * M * K) nonzero values

data_storage, format_storage = [], []

print(f'{"Density":>8} {"Data":>6} {"Format":>8} {"Combined":>10} '
      f'{"SL Data":>8} {"SL Fmt":>8} {"Match?":>8}')
print('-' * 62)

for i, da in enumerate(DENSITIES_PART4):
    # Data storage = total expected nonzeros
    data = math.ceil(da * M * K)
    
    # Format storage:
    # UOP offset array: M+1 = 9 entries
    # CP coordinates: M fibers * ceil(d * K) nonzeros per fiber
    uop_payload = M + 1  # 9
    ennz_per_fiber = math.ceil(da * K)
    cp_metadata = M * ennz_per_fiber
    fmt = uop_payload + cp_metadata
    
    data_storage.append(data)
    format_storage.append(fmt)
    
    match = 'Y' if data == SL_DATA[i] and fmt == SL_FORMAT[i] else 'N'
    print(f'{da:8.1f} {data:6d} {fmt:8d} {data+fmt:10d} '
          f'{SL_DATA[i]:8d} {SL_FORMAT[i]:8d} {match:>8}')

print(f'\nUncompressed A: {M * K} words')
print(f'Compression beneficial below density ~0.4 (where combined < {M*K})')

In [ ]:
# Plot buffer capacity vs density (matching Lab 4 Fig)
cap_df = pd.DataFrame({
    'density': DENSITIES_PART4 * 3,
    'type': ['data'] * 5 + ['format'] * 5 + ['combined'] * 5,
    'storage (words)': data_storage + format_storage + 
                       [d + f for d, f in zip(data_storage, format_storage)],
})

fig, ax = plt.subplots(figsize=(7, 4))
for typ, marker, color in [('data', 'o', 'tab:blue'), ('format', 's', 'tab:orange'),
                            ('combined', '^', 'tab:green')]:
    sub = cap_df[cap_df['type'] == typ]
    ax.plot(sub['density'], sub['storage (words)'], f'{marker}-', label=typ, color=color, linewidth=2)

ax.axhline(y=64, color='red', linestyle='--', alpha=0.7, label='Uncompressed (64)')
ax.set_xlabel('Density of A', fontsize=12)
ax.set_ylabel('Storage (words)', fontsize=12)
ax.set_title('Q4.1: Buffer Capacity for Tensor A (CSR Format)', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('Q4.2 Answers:')
print(f'  Uncompressed A: {M*K} words')
print(f'  Compression beneficial below density: 0.4')

---

# Part 5: Breaking Assumptions

The previous sections assumed independent distribution: P(effectual) = d_A * d_B.
Real data can violate this. We examine three patterns where the actual number of
effectual operations differs from the independent prediction.

In [ ]:
print('=== Q5.1: Identity Matrix ===')
# A = B = I_8 (identity matrix)
density = 8 / 64  # 0.125
independent_prediction = int(M**3 * density * density)  # 512 * 0.125 * 0.125 = 8
actual_effectual = M  # 8 (only diagonal elements multiply)
print(f'Density of A (and B): {density}')
print(f'Independent prediction: {independent_prediction} effectual MACs')
print(f'Actual effectual MACs: {actual_effectual}')
print(f'Match? {independent_prediction == actual_effectual} (coincidence!)')

print()
print('=== Q5.2: Column-Row Pattern ===')
# A: first column all ones. B: first row all ones.
# A[:,0] = 1 => for each m, only k=0 is nonzero
# B[0,:] = 1 => for k=0, all n are nonzero
# Every (m, k=0, n) is effectual => M * N = 64
density_cr = 8 / 64
independent_cr = int(M**3 * density_cr * density_cr)
actual_cr = M * N  # 64
print(f'Density of A (and B): {density_cr}')
print(f'Independent prediction: {independent_cr} effectual MACs')
print(f'Actual effectual MACs: {actual_cr}')
print(f'Much more than predicted! Nonzeros are correlated.')

print()
print('=== Q5.3: Modified Column-Row Pattern ===')
# A: first column all ones (k=0). B: LAST row all ones (k=7).
# A has nonzeros at k=0, B has nonzeros at k=7.
# No overlap in k => ZERO effectual operations!
actual_mcr = 0
print(f'Density of each matrix: {density_cr}')
print(f'Independent prediction: {independent_cr} effectual MACs')
print(f'Actual effectual MACs: {actual_mcr}')
print(f'Worst case: nonzeros are anti-correlated in the K dimension.')

---

# Extended Analysis: Energy and Latency Trends

Beyond the Lab 4 questions, we can explore how AccelForge models the energy-latency
tradeoffs across different optimizations and density levels.

In [ ]:
# Density sweep: compare dense, gating, skipping
DENSITIES_SWEEP = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

dense_e_sweep, gate_e_sweep, skip_e_sweep = [], [], []
dense_c_sweep, gate_c_sweep, skip_c_sweep = [], [], []

print(f'{"d_A":>5} {"Dense E":>10} {"Gate E":>10} {"Skip E":>10} '
      f'{"Dense C":>8} {"Gate C":>8} {"Skip C":>8}')
print('-' * 68)

for da in DENSITIES_SWEEP:
    # Keep d_B = 0.5 fixed
    rd = run_lab4(density_a=da, density_b=0.5)
    rg = run_lab4('sparse_gating.yaml', density_a=da, density_b=0.5)
    rs = run_lab4('sparse_skipping.yaml', density_a=da, density_b=0.5)
    
    de, dc = get_energy(rd), get_cycles(rd)
    ge, gc = get_energy(rg), get_cycles(rg)
    se, sc = get_energy(rs), get_cycles(rs)
    
    dense_e_sweep.append(de)
    gate_e_sweep.append(ge)
    skip_e_sweep.append(se)
    dense_c_sweep.append(dc)
    gate_c_sweep.append(gc)
    skip_c_sweep.append(sc)
    
    print(f'{da:5.1f} {de:10.2f} {ge:10.2f} {se:10.2f} '
          f'{dc:8.0f} {gc:8.0f} {sc:8.0f}')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Energy plot
ax1.plot(DENSITIES_SWEEP, dense_e_sweep, 'o-', label='Dense', color='gray', linewidth=2)
ax1.plot(DENSITIES_SWEEP, gate_e_sweep, 's-', label='Gating', color='tab:blue', linewidth=2)
ax1.plot(DENSITIES_SWEEP, skip_e_sweep, '^-', label='Skipping (CSR)', color='tab:green', linewidth=2)
ax1.set_xlabel('Density of A (d_B=0.5)', fontsize=12)
ax1.set_ylabel('Total Energy (pJ)', fontsize=12)
ax1.set_title('Energy vs Density', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Latency plot
ax2.plot(DENSITIES_SWEEP, dense_c_sweep, 'o-', label='Dense', color='gray', linewidth=2)
ax2.plot(DENSITIES_SWEEP, gate_c_sweep, 's-', label='Gating', color='tab:blue', linewidth=2)
ax2.plot(DENSITIES_SWEEP, skip_c_sweep, '^-', label='Skipping (CSR)', color='tab:green', linewidth=2)
ax2.set_xlabel('Density of A (d_B=0.5)', fontsize=12)
ax2.set_ylabel('Total Latency (cycles)', fontsize=12)
ax2.set_title('Latency vs Density', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

fig.suptitle('Lab 4: Energy and Latency vs Density (d_B=0.5 fixed)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Energy savings ratio
gate_savings = [(1 - ge/de) * 100 for ge, de in zip(gate_e_sweep, dense_e_sweep)]
skip_savings = [(1 - se/de) * 100 for se, de in zip(skip_e_sweep, dense_e_sweep)]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(DENSITIES_SWEEP, gate_savings, 's-', label='Gating savings', color='tab:blue', linewidth=2)
ax.plot(DENSITIES_SWEEP, skip_savings, '^-', label='Skipping savings', color='tab:green', linewidth=2)
ax.set_xlabel('Density of A (d_B=0.5)', fontsize=12)
ax.set_ylabel('Energy Savings vs Dense (%)', fontsize=12)
ax.set_title('Energy Savings from Sparse Optimizations', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('Key findings:')
print('  - Gating saves energy at all densities (more savings at lower density)')
print('  - Skipping saves more energy than gating (reduces data accesses + compute)')
print('  - Both savings decrease as density approaches 1.0 (fewer ineffectual ops)')
print('  - Gating has NO latency impact; Skipping reduces latency proportionally')

---

# Summary

## Lab 4 Key Results

| Question | Topic | Answer |
|----------|-------|--------|
| 1.1 | Loop order | MKN |
| 1.2 | Effectual (d_A=1, d_B=1) | 512 |
| 1.3 | Effectual (d_A=0.5, d_B=1) | 256 effectual, 256 ineffectual |
| 1.4 | Effectual (d_A=0.5, d_B=0.5) | 128 |
| 1.5 | Gating saves | Energy only |
| 1.5 | Skipping saves | Both energy + latency |
| 1.7 | Compression overhead | False (can exceed savings at high density) |
| 2.1 | Gating component | Buffer (storage), MAC (compute) |
| 2.1 | Unaffected component | DRAM |
| 2.1 | Latency impact | No impact |
| 3.2 | More sparsity + gating/skipping | Decreases total energy |
| 3.2 | More sparsity + skipping fJ/compute | Increases |
| 3.2 | More sparsity + skipping fJ/alg-compute | Decreases |
| 4.2 | Uncompressed A | 64 words |
| 4.2 | Compression beneficial below | ~0.4 density |
| 5.1 | Identity: predicted vs actual | 8 vs 8 (coincidence) |
| 5.2 | Column-row: predicted vs actual | 8 vs 64 |
| 5.3 | Modified column-row: actual | 0 (worst case) |

## AccelForge vs Sparseloop

ERT values from Accelergy (SRAM_metadata + regfile_metadata at 45nm) and corrected
`bits_per_action` (BackingStorage=32 matching DRAM width, Buffer=8 matching regfile width).

| Config | AccelForge (pJ) | Sparseloop (pJ) | Delta |
|--------|----------------|-----------------|-------|
| Dense | ~3601 | 3608 | -0.2% |
| Gating | ~2058 | 2034 | +1.2% |
| Skipping | ~1087 | 983 | +10.6% |

- **Dense and gating match within ~1%** of Sparseloop
- **Skipping ~11% off** due to metadata model differences (statistical vs simulation)
- **Trends match:** gating saves energy only, skipping saves energy+latency
- **Buffer capacity:** analytical CSR model **exactly matches** Sparseloop (all 5 density points)